# Segunda seccion de dudas:

## Modelo de Recomendación:
* Vectorización de Características
* Similitud del Coseno (cosine_similarity)
* Lógica de Filtrado y Priorización (Customizada)
* Visualización de Datos (Matplotlib)

### # Vectorización de Características

**1. Teoría: ¿Qué es la Vectorización de Características y por qué es el paso final crucial?**

**Concepto:**

La **vectorización de características** es el proceso de transformar todos los datos relevantes de una entidad (en tu caso, un Pokémon o las preferencias de un usuario) en una **representación numérica en forma de vector**. Este vector es una lista ordenada de números donde cada número corresponde a una característica específica y predefinida. Es, en esencia, la traducción de toda la información de un Pokémon a un "lenguaje" matemático que los algoritmos pueden procesar.

Piensa en ello como crear una "ficha numérica" estandarizada para cada Pokémon. Cada número en esa ficha representa una característica particular (ej., HP estandarizado, un 1 si es de tipo 'Water', un 0 si no lo es, etc.).

**¿Por qué es el paso final crucial?**

Los algoritmos de Machine Learning, especialmente aquellos que calculan distancias o similitudes (como la similitud del coseno), no pueden trabajar directamente con texto (como 'Grass', 'Fire') ni con valores numéricos en escalas muy diferentes. Necesitan que todos los datos estén en un formato numérico unificado y estandarizado.

La vectorización de características es el punto donde todos los pasos de preprocesamiento anteriores convergen:

1.  **Manejo de Nulos (`fillna`):** Asegura que no haya "huecos" en los datos que impidan una representación completa en el vector. Al rellenar `NaN` en `Type 2` con `'None'`, `'None'` se convierte en una categoría que puede ser vectorizada.
2.  **Codificación One-Hot (`pd.get_dummies`):** Transforma las características categóricas nominales (como 'Type 1', 'Type 2') en el formato binario de 0s y 1s, que son números y pueden ser parte de un vector. Cada categoría única se convierte en una dimensión del vector.
3.  **Normalización (`StandardScaler`):** Escala las características numéricas (HP, Attack, etc.) a un rango común (media 0, desviación estándar 1). Esto asegura que ninguna estadística domine el vector solo por tener valores más grandes, permitiendo que cada característica contribuya equitativamente a la similitud.

El resultado de la vectorización es que cada Pokémon (y el conjunto de preferencias del usuario) se representa como un único vector en un espacio de muchas dimensiones.

**Ejemplo Ilustrativo:**

Imagina que después de todo el preprocesamiento, tenemos las siguientes características combinadas para un Pokémon:

* **Estadísticas Estandarizadas:** `HP_scaled`, `Attack_scaled`, `Defense_scaled`, `Sp. Atk_scaled`, `Sp. Def_scaled`, `Speed_scaled`.
* **Tipos One-Hot:** `Type1_Grass`, `Type1_Poison`, `Type1_Fire`, `Type2_None`, `Type2_Flying`, etc.

Para un Pokémon como Bulbasaur (Grass/Poison, con ciertas estadísticas):

$$ \text{Vector de Bulbasaur} = [ \underbrace{HP_{\text{scaled}}, Attack_{\text{scaled}}, \dots}_{\text{Estadísticas estandarizadas}}, \underbrace{1_{\text{Grass}}, 0_{\text{Fire}}, \dots, 1_{\text{Poison}}, 0_{\text{None}}, \dots}_{\text{Tipos One-Hot}} ] $$

Cada elemento en este vector es un número. Las estadísticas serán valores continuos (positivos o negativos) y los tipos serán 0s o 1s. Todos estos números juntos forman el vector de características para Bulbasaur.

**Importancia para la Similitud del Coseno:**

La similitud del coseno opera directamente sobre estos vectores. Toma dos vectores (por ejemplo, el vector de preferencias del usuario y el vector de un Pokémon) y calcula el coseno del ángulo entre ellos. Un ángulo pequeño (coseno cercano a 1) significa que los vectores "apuntan" en direcciones similares en el espacio multidimensional, indicando una alta similitud.

Sin la vectorización, sería imposible cuantificar la "similitud" de manera matemática entre entidades que tienen tipos de datos tan diferentes (texto, números con diferentes escalas). La vectorización proporciona la representación numérica uniforme necesaria para que el algoritmo trabaje.

---

**2. Implementación en Código Python**

La vectorización de características no es una única línea de código con un método `vectorize()`. En tu proyecto, es la **culminación de los pasos de preprocesamiento** aplicados al DataFrame `df_processed`. El resultado final de esos pasos es el DataFrame `pokemon_features_df`, donde cada fila es el vector de características de un Pokémon.

```python
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# --- Re-creamos un escenario simplificado de cómo se llega a la vectorización ---

# 1. Suponemos df ya cargado y Type 2 rellenado con 'None'
# df = pd.read_csv('pkmn.csv')
# df['Type 2'] = df['Type 2'].fillna('None')

# 2. Creamos df_processed (la copia que será vectorizada)
df_processed = df.copy()

# 3. Aplicamos One-Hot Encoding a las columnas de tipo
df_processed = pd.get_dummies(
    df_processed,
    columns=['Type 1', 'Type 2'],
    prefix=['Type1', 'Type2'],
    dummy_na=False
)

# 4. Definimos las columnas numéricas para escalar
numeric_cols = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

# 5. Inicializamos y aplicamos StandardScaler a las columnas numéricas
scaler = StandardScaler()
df_processed[numeric_cols] = scaler.fit_transform(df_processed[numeric_cols])

# --- El paso final de vectorización para el algoritmo ---
# Ahora, seleccionamos las columnas de df_processed que formarán el vector de características.
# Excluimos columnas como '#', 'Name' ya que no son características numéricas para la similitud.

# Identificar todas las columnas que son características (numéricas escaladas o binarias de tipos)
# Esto se hace seleccionando todas las columnas excepto aquellas que no son características útiles para la similitud.
# Asumiendo que 'Name' y '#' no son características que quieres usar en el cálculo de similitud.
columns_to_include_in_features = [col for col in df_processed.columns if col not in ['#', 'Name']]

# pokemon_features_df es el DataFrame donde cada fila es un vector de características de un Pokémon
pokemon_features_df = df_processed[columns_to_include_in_features]

# --- Verificamos el resultado de la vectorización ---
# print("\nDataFrame de Características Vectorizadas (primeras 5 filas y algunas columnas):")
# print(pokemon_features_df.head())
# print(f"\nNúmero total de características (dimensiones del vector): {pokemon_features_df.shape[1]}")

### # Lógica de Priorización: Dando más Importancia a los Tipos en la Similitud

**1. Teoría: ¿Por qué priorizar ciertas características y cómo se hace en este proyecto?**

**Concepto de Priorización:**

En un sistema de recomendación, a menudo no todas las características tienen la misma importancia para el usuario o para el objetivo del sistema. Aunque la similitud del coseno calcula una "distancia" o "afinidad" general entre vectores, a veces queremos que ciertas características (como los tipos en tu caso) tengan un peso o una influencia desproporcionadamente mayor en las recomendaciones finales. Esto es lo que llamamos **priorización** o **ponderación**.

**¿Por qué es importante en tu proyecto de Pokémon?**

Para un fan de Pokémon, el **tipo** es una de las características más fundamentales. Un usuario que busca un Pokémon de tipo "Agua" probablemente lo considere mucho más importante que una ligera diferencia en "HP" o "Defense". Si solo te basaras en la similitud del coseno pura sobre todas las características (tipos y estadísticas), un Pokémon de tipo "Fuego" con estadísticas muy similares a las deseadas por el usuario podría aparecer antes que un Pokémon de tipo "Agua" con estadísticas ligeramente menos similares. La priorización de tipos corrige esto.

**Métodos Comunes de Priorización:**

Existen varias formas de dar más importancia a ciertas características:

1.  **Ponderación Directa de Características (antes de la similitud):** Multiplicar las columnas de características por un factor de peso antes de calcular la similitud. Esto aumenta su "longitud" o "activación" en el espacio vectorial.
2.  **Lógica de Negocio Post-Similitud (lo que haces tú):** Calcular la similitud general primero, y luego aplicar reglas de filtrado o reordenamiento basadas en las características prioritarias.
3.  **Modelos de Clasificación/Regresión (más complejos):** Usar un modelo supervisado para aprender la importancia de las características a partir de datos históricos de preferencias.

En tu proyecto, la estrategia utilizada es la **lógica de negocio post-similitud**. Esto significa que la `cosine_similarity` se calcula primero sobre el vector completo (que incluye tanto tipos One-Hot como estadísticas estandarizadas). Sin embargo, después de obtener esa similitud general, el algoritmo aplica un **filtro y reordenamiento manual** para asegurar que los Pokémon que coinciden con los tipos primarios y secundarios del usuario sean priorizados.

**El Proceso de Priorización en tu Código:**

Tu código sigue una secuencia lógica para asegurar que los tipos sean el factor dominante:

1.  **Similitud General:** Primero, se calcula la `cosine_similarity` entre el vector de preferencias del usuario y los vectores de todos los Pokémon. Esto te da una lista de índices de Pokémon ordenados por su similitud general (que ya considera tipos y estadísticas).
2.  **Filtrado por `Type 1`:** El algoritmo itera sobre los Pokémon ya ordenados por similitud general. La primera prioridad es encontrar Pokémon cuyo `Type 1` (tipo primario) **coincida exactamente** con el `user_type1` proporcionado. Estos Pokémon se guardan en una lista temporal.
3.  **Filtrado y Priorización por `Type 2` (si aplica):**
    * Si el usuario también especificó un `user_type2`, el algoritmo toma los Pokémon que ya coincidieron con `Type 1` y los divide: aquellos que también coinciden con `user_type2` (doble coincidencia de tipo) y aquellos que solo coinciden con `Type 1`.
    * Los Pokémon con **doble coincidencia de tipo** se añaden a la lista final de recomendaciones **primero**, hasta alcanzar el número `top_n` deseado.
    * Si aún faltan recomendaciones, se añaden los que solo coincidieron con `Type 1`.
4.  **Relleno con Similitud General:** Si, después de todas las priorizaciones por tipo, todavía no se ha alcanzado el número `top_n` de recomendaciones, el algoritmo "rellena" la lista con los siguientes Pokémon más similares de la lista general (que no hayan sido ya incluidos), asegurando que siempre se devuelva el número deseado de recomendaciones.

Este enfoque asegura que los Pokémon que cumplen con los criterios de tipo primario (y secundario, si se especificó) obtengan una preferencia explícita en las recomendaciones finales, incluso si sus estadísticas no los colocaron en la cima absoluta por similitud general.

---

**2. Implementación en Código Python**

Este proceso se implementa directamente en la función `recommend_pokemons` y es la parte que va *después* del cálculo de la `cosine_similarity`.

```python
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import os
import base64
import streamlit as st

# ... (asumimos que df, df_processed, scaler, numeric_cols ya están cargados y preprocesados) ...

# La función recommend_pokemons, con énfasis en la lógica de priorización:
def recommend_pokemons(df, df_processed, scaler, numeric_cols, user_type1, user_type2=None, user_hp=None, user_attack=None, user_defense=None, user_sp_atk=None, user_sp_def=None, user_speed=None, top_n=3):
    # 1. Preparación del vector de características del usuario (ESTANDARIZADO)
    # ... (código para crear user_features) ...

    # 2. Preparación del DataFrame de características de Pokémon (ESTANDARIZADO)
    columns_to_include_in_features = [col for col in df_processed.columns if col not in ['#', 'Name']]
    pokemon_features_df = df_processed[columns_to_include_in_features]

    # 3. Calcular la similitud del coseno (se realiza sobre los vectores estandarizados)
    # user_features es un array 2D, pokemon_features_df es un DataFrame
    similarities = cosine_similarity(user_features, pokemon_features_df)

    # Obtener los índices de los Pokémon más similares (ordenados descendentemente por similitud)
    similar_pokemon_internal_indices = similarities.argsort()[0][::-1]

    # 4. === LÓGICA DE PRIORIZACIÓN DE TIPOS ===
    final_recommended_internal_indices = []

    # PASO A: Primero, identificar Pokémon que coinciden con user_type1
    temp_type1_matches = []
    for internal_idx in similar_pokemon_internal_indices:
        # Usamos el DataFrame ORIGINAL 'df' para verificar el tipo, ya que es más legible
        if df.loc[internal_idx, 'Type 1'] == user_type1:
            temp_type1_matches.append(internal_idx)

    # PASO B: Si se especificó Type 2, priorizar aún más
    if user_type2:
        temp_type1_type2_matches = [] # Coinciden con Type 1 y Type 2
        temp_type1_only_matches = []  # Solo coinciden con Type 1
        
        for internal_idx in temp_type1_matches:
            if df.loc[internal_idx, 'Type 2'] == user_type2:
                temp_type1_type2_matches.append(internal_idx)
            else:
                temp_type1_only_matches.append(internal_idx)

        # 1. Agregamos primero los Pokémon que coinciden con AMBOS tipos, hasta top_n
        final_recommended_internal_indices.extend(temp_type1_type2_matches[:top_n])

        # 2. Si aún faltan recomendaciones, agregamos los que solo coinciden con Type 1
        if len(final_recommended_internal_indices) < top_n:
            remaining_needed = top_n - len(final_recommended_internal_indices)
            final_recommended_internal_indices.extend(temp_type1_only_matches[:remaining_needed])

    else: # Si user_type2 NO se especifica, solo toma los mejores por Type 1
        final_recommended_internal_indices.extend(temp_type1_matches[:top_n])

    # PASO C: Si aún no hemos alcanzado top_n, agregamos los Pokémon más similares en general
    # (esto cubre casos donde no hay suficientes coincidencias de tipo o si se quieren más)
    if len(final_recommended_internal_indices) < top_n:
        remaining_needed = top_n - len(final_recommended_internal_indices)
        count_added = 0
        for internal_idx in similar_pokemon_internal_indices:
            # Asegurarse de no añadir duplicados
            if internal_idx not in final_recommended_internal_indices:
                final_recommended_internal_indices.append(internal_idx)
                count_added += 1
                if count_added >= remaining_needed:
                    break
    
    # Asegurarse de que no haya duplicados y limitar a top_n (una verificación final)
    final_recommended_internal_indices = list(dict.fromkeys(final_recommended_internal_indices))[:top_n]


    # 5. Obtener el DataFrame de los Pokémon recomendados usando los índices finales
    # NOTA CLAVE: se usa el DataFrame 'df' (el ORIGINAL, NO ESTANDARIZADO) para la presentación
    recommended_pokemons = df.loc[final_recommended_internal_indices].copy()

    # Añadir la columna de Similitud para información (opcional, pero útil)
    recommended_pokemons['Similarity'] = [similarities[0][idx] for idx in final_recommended_internal_indices]

    return recommended_pokemons

### # Consistencia de la Codificación One-Hot: Usuario vs. DataFrame

**Tu pregunta:** "¿Cómo el algoritmo se asegura que los `Type 1` y `Type 2` insertados por el usuario sigan la misma lógica que los del DataFrame?"

Esta es una preocupación muy válida y aborda un punto fundamental en el preprocesamiento: la **consistencia de la transformación**. Si los tipos del usuario se codifican de una manera diferente a como se codificaron los tipos de los Pokémon en el dataset, el cálculo de la similitud del coseno no tendría sentido (los vectores no estarían en el mismo "espacio" de características).

La garantía de esta consistencia reside en el objeto `pd.get_dummies` y cómo se aplica el `StandardScaler`.

**1. El "Aprendizaje" del `pd.get_dummies` (Implícito):**

Aunque `pd.get_dummies` no es un objeto que "aprende" como `StandardScaler` (que tiene un `.fit()` y un `.transform()`), su comportamiento es consistente si se le aplica a los mismos datos de entrada o a datos que contengan el mismo conjunto de categorías.

* **Paso 1: Codificación de `df_processed` (el dataset completo):**
    Cuando se ejecuta `df_processed = pd.get_dummies(df_processed, columns=['Type 1', 'Type 2'], prefix=['Type1', 'Type2'], dummy_na=False)`, Pandas examina las columnas 'Type 1' y 'Type 2' de *todo el DataFrame de Pokémon*.
    * Identifica **todas las categorías únicas** presentes en 'Type 1' (ej., 'Grass', 'Fire', 'Water', 'Electric', ...) y en 'Type 2' (ej., 'Poison', 'Flying', 'Ground', 'None', ...).
    * Para cada una de estas categorías únicas, crea una **nueva columna binaria** (ej., `Type1_Grass`, `Type1_Fire`, `Type2_Poison`, `Type2_None`). La **posición** de estas columnas en el DataFrame resultante es consistente.

* **Paso 2: Codificación de las Preferencias del Usuario:**
    Cuando el usuario introduce sus preferencias, estas se colocan en un pequeño DataFrame temporal (`user_df`). Luego, para asegurar la coherencia, este `user_df` debe ser procesado de la siguiente manera:
    * **Método para asegurar la consistencia:** En lugar de llamar a `pd.get_dummies` directamente en `user_df` de forma aislada, que podría crear un conjunto diferente de columnas si el usuario no introduce todos los tipos posibles, la práctica estándar es:
        * Obtener la lista de todas las columnas de tipos One-Hot generadas en `df_processed`.
        * Crear un DataFrame "vacío" o "de referencia" para el usuario que tenga *todas esas mismas columnas*.
        * Rellenar con '1' las columnas correspondientes a los tipos del usuario y con '0' el resto.

    En tu código, esto se logra de una forma muy eficiente, aunque el `pd.get_dummies` no se llama directamente sobre `user_df`. En lugar de eso, el vector `user_features` se construye **mapeando las preferencias del usuario directamente a las columnas existentes** que ya se generaron en `pokemon_features_df`.

    * Las columnas de `pokemon_features_df` (que se crean a partir del `df_processed` completo) representan todas las posibles dimensiones de tipos y estadísticas.
    * Cuando se construye `user_features`, se inicializa un array de ceros del mismo tamaño que las columnas de `pokemon_features_df`.
    * Luego, se "activan" (se pone un '1') las posiciones correspondientes a los tipos (`Type1_X`, `Type2_Y`) que el usuario seleccionó.

    **Fórmula conceptual:**
    $$ \text{user\_vector}[i] = \begin{cases} 1 & \text{si la característica } i \text{ coincide con la preferencia del usuario} \\ 0 & \text{si la característica } i \text{ no coincide o no está presente en la preferencia del usuario} \end{cases} $$

    Esto garantiza que el vector del usuario tenga exactamente las mismas dimensiones y el mismo orden de características que los vectores de los Pokémon en tu dataset.

**2. El "Aprendizaje" del `StandardScaler`:**

El `StandardScaler` es un objeto que "aprende" de forma explícita y luego "transforma" de forma consistente:

* **Paso 1: `.fit()` en `df_processed`:**
    Cuando se ejecuta `scaler = StandardScaler()` y luego `scaler.fit_transform(df_processed[numeric_cols])`, el método `.fit()` del `StandardScaler` calcula la **media ($\mu$)** y la **desviación estándar ($\sigma$)** para *cada columna numérica* (HP, Attack, Defense, etc.) basándose en *todos los datos de Pokémon*. Estos valores ($\mu$ y $\sigma$) se "guardan" dentro del objeto `scaler`.

* **Paso 2: `.transform()` para las Preferencias del Usuario:**
    Cuando el usuario introduce sus valores numéricos (ej., HP=100, Attack=80), estos valores **no se estandarizan con una nueva media y desviación estándar**. En su lugar, se usa el *mismo* objeto `scaler` que ya fue "entrenado" con el dataset completo.
    * Se utiliza el método `scaler.transform(user_df[numeric_cols])`.
    * Esto aplica la fórmula $z = \frac{x - \mu_{\text{dataset}}}{\sigma_{\text{dataset}}}$ a los valores del usuario, usando las $\mu$ y $\sigma$ que el `scaler` aprendió del *dataset completo de Pokémon*.

    **Fórmula de estandarización para usuario:**
    $$ z_{\text{usuario}} = \frac{x_{\text{usuario}} - \mu_{\text{dataset}}}{\sigma_{\text{dataset}}} $$

**Conclusión Clave:**

La consistencia en la vectorización de características, tanto para tipos como para estadísticas, se asegura porque:

1.  **Las columnas para los tipos One-Hot** en el vector del usuario son exactamente las mismas columnas (en número y orden) que se generaron para todo el dataset de Pokémon. No se crean nuevas columnas si el usuario introduce un tipo que no está en el dataset, ni se omiten columnas si el usuario no introduce un tipo.
2.  **Las estadísticas numéricas del usuario se estandarizan** utilizando la *misma media y desviación estándar* que se calcularon a partir de *todo el dataset de Pokémon*.

Esto garantiza que el vector de preferencias del usuario y los vectores de los Pokémon residan en el mismo espacio multidimensional, permitiendo una comparación significativa a través de la similitud del coseno.

---

### # Resumen Final de la Lógica del Recomendador de Pokémon (Confirmación de Entendimiento)

¡Sí, lo has entendido de forma **excelente**! Tu resumen captura la esencia del proceso completo. Desglosemos tu entendimiento para confirmarlo en detalle:

**Tu Afirmación:** "El usuario a medida que pone características (está construyendo un vector)"

* **Confirmación:** ¡Correcto! Cada vez que el usuario selecciona un `Type 1`, `Type 2` y/o ingresa valores para las estadísticas (HP, Attack, etc.), el sistema está construyendo un **vector de preferencias del usuario**. Este vector se somete al mismo preprocesamiento que los Pokémon del dataset: los tipos se codifican One-Hot y las estadísticas numéricas se estandarizan utilizando el mismo `StandardScaler` que se ajustó a los datos de todos los Pokémon. Esto asegura que el vector del usuario esté en el mismo "espacio" y escala que los vectores de los Pokémon.

**Tu Afirmación:** "Este vector se compara a través de la similitud del coseno y se buscan los vectores más similares"

* **Confirmación:** ¡Exactamente! Una vez que el vector de preferencias del usuario está construido y estandarizado, se utiliza la **similitud del coseno** para compararlo con el vector de características de *cada uno* de los Pokémon en tu dataset (`pokemon_features_df`). La similitud del coseno es ideal porque mide la "dirección" (afinidad conceptual) entre los vectores en el espacio multidimensional de características, ignorando las magnitudes absolutas que podrían surgir de diferencias en el número de "1"s en los tipos One-Hot o en la escala de las estadísticas. El resultado es una puntuación de similitud para cada Pokémon.

**Tu Afirmación:** "también se filtra por tipo para darle prioridad (más peso) y así tener los 3 Pokémon recomendados óptimamente"

* **Confirmación:** ¡Precisamente! Este es el paso crucial de la "lógica de negocio" o "priorización". Aunque la similitud del coseno ya considera los tipos como parte del vector de características, el sistema va un paso más allá para asegurar que las coincidencias de tipo sean un factor dominante en las recomendaciones finales.
    * Después de obtener el ranking inicial de similitud de coseno, el algoritmo **filtra y reordena activamente** las recomendaciones.
    * Primero busca Pokémon que coincidan con el `Type 1` del usuario.
    * Si se especificó un `Type 2`, prioriza aún más a aquellos que coinciden con *ambos* tipos.
    * Este proceso garantiza que los Pokémon que cumplen con los criterios de tipo sean fuertemente favorecidos en la lista de los `top_n` (en tu caso, 3) recomendados, incluso si un Pokémon de un tipo "incorrecto" hubiera tenido una similitud de coseno ligeramente más alta debido a una coincidencia estadística perfecta.

**En resumen, tu entendimiento es impecable.** El sistema funciona de la siguiente manera:

1.  **Entrada del Usuario:** Se traduce a un vector de preferencias numéricas y estandarizadas.
2.  **Preparación de Datos:** Todos los Pokémon del dataset se transforman en vectores de características numéricas y estandarizadas (One-Hot para tipos, `StandardScaler` para estadísticas).
3.  **Cálculo de Afinidad:** La similitud del coseno se usa para cuantificar cuán cerca está el vector del usuario de cada vector de Pokémon.
4.  **Ajuste de Prioridad:** Se aplica una lógica de filtrado y reordenamiento manual basada en las coincidencias de `Type 1` y `Type 2` para asegurar que estos criterios sean los más influyentes en las recomendaciones finales.
5.  **Output:** Se presentan los 3 Pokémon más relevantes, recuperando sus datos originales (no estandarizados) para una mejor comprensión.

¡Felicidades, has integrado muy bien todos los conceptos!

---